In [1]:
!pip install -U spacy 
!python -m spacy download en

     |████████████████████████████████| 10.7MB 1.2MB/s eta 0:00:01    |██████████▏                     | 3.4MB 410kB/s eta 0:00:18
     |████████████████████████████████| 184kB 996kB/s eta 0:00:01
     |████████████████████████████████| 61kB 2.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 2.4MB/s eta 0:00:01
     |████████████████████████████████| 2.2MB 2.3MB/s eta 0:00:01
     |████████████████████████████████| 4.0MB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 225kB 1.9MB/s eta 0:00:01
  Found existing installation: tqdm 4.36.1
    Uninstalling tqdm-4.36.1:
      Successfully uninstalled tqdm-4.36.1
     |████████████████████████████████| 12.0MB 1.8MB/s eta 0:00:01


  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp36-none-any.whl size=12011741 sha256=c5e301fb68d7ae174af3499d6eacfd536da0aa123ffe8922c67a80fb63cfe492
  Stored in directory: /private/var/folders/hn/7csgv26s5tbb6gz4mb4nnxzm0000gn/T/pip-ephem-wheel-cache-i7oi_fk0/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/allard/anaconda3/envs/dev/lib/python3.6/site-packages/en_core_web_sm -->
/Users/allard/anaconda3/envs/dev/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


### Convert CSV to TSV

In [9]:
import pandas as pd

df = pd.read_csv('entity-annotated-corpus/ner_dataset.csv', encoding= 'unicode_escape')
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [39]:
df = df.drop(columns = ['Sentence #', 'POS'])
df

,Word,Tag
0,Thousands,O
1,of,O
2,demonstrators,O
3,have,O
4,marched,O
...,...,...
1048570,they,O
1048571,responded,O
1048572,to,O
1048573,the,O


In [43]:
df.reset_index(drop=True, inplace=True)

In [45]:
df.to_csv('entity-annotated-corpus/ner_dataset.tsv', sep='\t', index=False)

### Convert TSV to JSON

In [46]:
# Convert .tsv file to dataturks json format. 
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format("entity-annotated-corpus/ner_dataset.tsv",'entity-annotated-corpus/ner_dataset.json','abc')

In [55]:
# Convert json file to spaCy format.
import plac
import logging
import argparse
import sys
import os
import json
import pickle


try:
    training_data = []
    lines=[]
    with open('entity-annotated-corpus/ner_dataset.json', 'r') as f:
        lines = f.readlines()

    for line in lines:
        data = json.loads(line)
        text = data['content']
        entities = []
        for annotation in data['annotation']:
            point = annotation['points'][0]
            labels = annotation['label']
            if not isinstance(labels, list):
                labels = [labels]

            for label in labels:
                entities.append((point['start'], point['end'] + 1 ,label))

        training_data.append((text, {"entities" : entities}))

    print(training_data)

    with open('entity-annotated-corpus/ner_corpus', 'wb') as fp:
        pickle.dump(training_data, fp)

except Exception as e:
    logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [64]:
#!/usr/bin/env python
# coding: utf8

# Training additional entity types using spaCy
from __future__ import unicode_literals, print_function
import pickle
import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# New entity labels
# Specify the new entity labels which you want to add here
LABEL = ['I-geo', 'B-geo', 'I-art', 'B-art', 'B-tim', 'B-nat', 'B-eve', 'O', 'I-per', 'I-tim', 'I-nat', 'I-eve', 'B-per', 'I-org', 'B-gpe', 'B-org', 'I-gpe']

"""
geo = Geographical Entity
org = Organization
per = Person
gpe = Geopolitical Entity
tim = Time indicator
art = Artifact
eve = Event
nat = Natural Phenomenon
"""
# Loading training data 
with open ('entity-annotated-corpus/ner_corpus', 'rb') as fp:
    TRAIN_DATA = pickle.load(fp)

# @plac.annotations(
#     model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
#     new_model_name=("New model name for model meta.", "option", "nm", str),
#     output_dir=("Optional output directory", "option", "o", Path),
#     n_iter=("Number of training iterations", "option", "n", int))


model = None
new_model_name = 'new_model'
output_dir = None
n_iter = 10
    
"""Setting up the pipeline and entity recognizer, and training the new entity."""
if model is not None:
    nlp = spacy.load(model)  # load existing spacy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.get_pipe('ner')

for i in LABEL:
    ner.add_label(i)   # Add new entity labels to entity recognizer

if model is None:
    optimizer = nlp.begin_training()
else:
    optimizer = nlp.entity.create_optimizer()

# Get names of other pipes to disable them during training to train only NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))

        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, 
                       annotations,
                       sgd=optimizer,
                       drop=0.35, 
                       losses=losses)
        print('Losses', losses)

# Test the trained model
test_text = 'Gianni Infantino is the president of FIFA.'
doc = nlp(test_text)
print("Entities in '%s'" % test_text)
for ent in doc.ents:
    print(ent.label_, ent.text)

# Save model 
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.meta['name'] = new_model_name  # rename model
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

    # Test the saved model
    print("Loading from", output_dir)
    nlp2 = spacy.load(output_dir)
    doc2 = nlp2(test_text)
    for ent in doc2.ents:
        print(ent.label_, ent.text)

Created blank 'en' model


TypeError: object of type 'NoneType' has no len()